# Add Bands 1 and 2 to database

The goal of this notebook is to add estimates of bands 1 and 2 to the database. I want to keep the original database code intact, so doing this in a separate notebook.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, QTable, vstack
import astropy.units as u
from importlib import reload
import wsu_plots
import wsu_db
import large_cubes

## reading in the data

In [3]:
# fix up custom units
gvis = u.def_unit('Gvis',namespace=globals())
u.add_enabled_units([gvis])

In [4]:
result_c7_mous = QTable.read('data/wsu_datarates_mit_per_mous_cycle7_20230919.ecsv')
result_c8_mous = QTable.read('data/wsu_datarates_mit_per_mous_cycle8_20230919.ecsv')

In [5]:
wsu_all = vstack([result_c7_mous,result_c8_mous])

In [6]:
wsu_all.columns

<TableColumns names=('mous','proposal_id','schedblock_name','array','science_keyword','scientific_category','scientific_category_proposal','nant_typical','nant_array','nant_all','band','ntarget','s_fov','s_resolution','mosaic','imsize','pb','cell','blc_npol','blc_nspw','blc_specwidth','blc_freq','blc_velres','blc_nchan_agg','blc_nchan_max','blc_bandwidth_max','blc_bandwidth_agg','wsu_freq','wsu_npol','wsu_bandwidth_early','wsu_bandwidth_later_2x','wsu_bandwidth_later_4x','wsu_bandwidth_spw','wsu_nspw_early','wsu_nspw_later_2x','wsu_nspw_later_4x','wsu_specwidth_finest','wsu_chanavg_finest','wsu_velres_finest','wsu_specwidth_stepped','wsu_chanavg_stepped','wsu_velres_stepped','wsu_specwidth_stepped2','wsu_chanavg_stepped2','wsu_velres_stepped2','wsu_tint','wsu_nchan_spw_finest','wsu_nchan_spw_stepped','wsu_nchan_spw_stepped2','wsu_frac_bw_early','wsu_frac_bw_later_2x','wsu_frac_bw_later_4x','wsu_frac_bw_spw','nbase_typical','nbase_array','nbase_all','L80','Project','SB_name','blc_tint',

## calculating the fraction of total time

In [10]:
idx = (wsu_all['array'] == '12m') 
total_time = np.nansum(wsu_all[idx]['time_tot']).to('hr')
print(total_time)

5227.821893333333 h


The total time each for band 1 and band 2 (lower end) is 5%.

In [12]:
total_time*0.05

<Quantity 261.39109467 h>

In [68]:
reload(wsu_db)
test = wsu_db.remove_projects(wsu_all,array='12m',time_frac=0.05)

Total number of MOUSes: 5200
Total number of MOUSes meeting the criteria: 2826
Number of MOUSes removed: 180
New number of MOUSes: 5020


In [71]:
reload(wsu_db)
test2 = wsu_db.remove_projects(test,array='7m',time_frac=0.05)

Total number of MOUSes: 5020
Total number of MOUSes meeting the criteria: 1493
Number of MOUSes removed: 102
New number of MOUSes: 4918


In [ ]:
reload(wsu_db)
wsu_db.add_bands(test2, array='12m', band=1.0,total_time=260.0*u.hr)

--Return--
None
> /Users/akepley/Dropbox/Support/naasc/WSU/big_cubes/wsu_db.py(1761)add_bands()
   1760 
-> 1761     ipdb.set_trace()
   1762 

ipdb> len(db_update)
158
ipdb> np.nansum(db_update['time_tot']).to('hr')
<Quantity 264.57256 h>
ipdb> dbupdate]
*** SyntaxError: unmatched ']'
ipdb> db_update['wsu_velres_stepped2']
<Quantity [9.96351581, 9.96136765, 9.97637523, 9.96496876, 1.98909794,
           1.99836654, 0.4996031 , 9.96326499, 9.97730162, 1.98871583,
           9.96247258, 9.97026072, 9.96136845, 0.49008751, 0.09052306,
           1.99423896, 0.48396842, 1.98092634, 9.9635394 , 9.96503277,
           0.08841284, 0.48679912, 1.96934504, 0.47060032, 0.08943599,
           1.99690147, 9.98184705, 9.9808087 , 0.4705997 , 9.96316011,
           9.96856614, 9.98441556, 9.96232653, 1.97387472, 9.96174261,
           0.47060022, 9.96469118, 1.97294132, 0.47060032, 1.9721144 ,
           0.47059648, 0.47060007, 0.07101546, 1.97265924, 0.48516134,
           1.99324981, 1.99883274, 

In [92]:
%debug

> /Users/akepley/opt/anaconda3/lib/python3.8/site-packages/astropy/table/table.py(3022)insert_row()
   3020 
   3021             if len(self.columns) != len(vals):
-> 3022                 raise ValueError('Mismatch between number of vals and columns')
   3023 
   3024             if mask is not None:

ipdb> up
> /Users/akepley/opt/anaconda3/lib/python3.8/site-packages/astropy/table/table.py(2943)add_row()
   2941              3   6   9
   2942         """
-> 2943         self.insert_row(len(self), vals, mask)
   2944 
   2945     def insert_row(self, index, vals=None, mask=None):

ipdb> up
> /Users/akepley/Dropbox/Support/naasc/WSU/big_cubes/wsu_db.py(1728)add_bands()
   1726 
   1727     ## need to convert total_time to seconds
-> 1728     while time_accum <= total_time.to('s'):
   1729         myrow_tmp = rng.integers(low=0,high=nrows,size=1)
   1730 

ipdb> time_accum
<Quantity 3211.488 s>
ipdb> total_time.to('s')
<Quantity 936000. s>
ipdb> len(db_update.columns)
141
ipdb> len(new_i

In [87]:
%debug

> /Users/akepley/opt/anaconda3/lib/python3.8/site-packages/astropy/table/table.py(3466)_rows_equal()
   3464                 # because the other array is not masked.
   3465                 false_mask = np.zeros(1, dtype=[(n, bool) for n in self.dtype.names])
-> 3466                 result = (self.as_array().data == other) & (self.mask == false_mask)
   3467         else:
   3468             if isinstance(other, np.ma.MaskedArray):

ipdb> time_accum
*** NameError: name 'time_accum' is not defined
ipdb> up
> /Users/akepley/opt/anaconda3/lib/python3.8/site-packages/astropy/table/table.py(3426)__eq__()
   3424 
   3425     def __eq__(self, other):
-> 3426         return self._rows_equal(other)
   3427 
   3428     def __ne__(self, other):

ipdb> up
> /Users/akepley/opt/anaconda3/lib/python3.8/site-packages/astropy/table/table.py(3429)__ne__()
   3427 
   3428     def __ne__(self, other):
-> 3429         return ~self.__eq__(other)
   3430 
   3431     def _rows_equal(self, other):

ipdb> u

In [69]:
len(test)

5020

In [72]:
len(test2)

4918

In [76]:
idx = (test2['array'] == '12m') & (test2['band'] == 3.0)
print(len(test2[idx]))
print(np.nansum(test2[idx]['time_tot']).to('hr'))

999
1468.404 h


In [79]:
test2['imsize','cell']

imsize,cell
,arcsec
float64,float64
220.0,2.483750395209291
245.0,2.2605336545695236
240.0,2.2827531148560913
225.0,2.4621422888842592
235.0,2.3507844575555152
225.0,2.446616204960742
390.0,0.9492306037505012
350.0,1.0683185835377622


In [39]:
%debug

> /Users/akepley/Dropbox/Support/naasc/WSU/big_cubes/wsu_db.py(1645)remove_projects()
   1643 
   1644     ## set up counters
-> 1645     time_accum = 0.0
   1646     myrows = np.array([])
   1647 

ipdb> myrows.append(myrow_tmp)
*** AttributeError: 'numpy.ndarray' object has no attribute 'append'
ipdb> ?np.extend
*** No help for 'np.extend'
ipdb> exit


In [15]:
rng = np.random.default_rng()

In [22]:
rng.integers(low=0,high=len(wsu_all))

4731

In [21]:
?np.random.default_rng().integers

Object `np.random.default_rng().integers` not found.


In [23]:
len(wsu_all)

5200

In [35]:
wsu_all[[0,100,500]][2]

mous,proposal_id,schedblock_name,array,science_keyword,scientific_category,scientific_category_proposal,nant_typical,nant_array,nant_all,band,ntarget,s_fov,s_resolution,mosaic,imsize,pb,cell,blc_npol,blc_nspw,blc_specwidth,blc_freq,blc_velres,blc_nchan_agg,blc_nchan_max,blc_bandwidth_max,blc_bandwidth_agg,wsu_freq,wsu_npol,wsu_bandwidth_early,wsu_bandwidth_later_2x,wsu_bandwidth_later_4x,wsu_bandwidth_spw,wsu_nspw_early,wsu_nspw_later_2x,wsu_nspw_later_4x,wsu_specwidth_finest,wsu_chanavg_finest,wsu_velres_finest,wsu_specwidth_stepped,wsu_chanavg_stepped,wsu_velres_stepped,wsu_specwidth_stepped2,wsu_chanavg_stepped2,wsu_velres_stepped2,wsu_tint,wsu_nchan_spw_finest,wsu_nchan_spw_stepped,wsu_nchan_spw_stepped2,wsu_frac_bw_early,wsu_frac_bw_later_2x,wsu_frac_bw_later_4x,wsu_frac_bw_spw,nbase_typical,nbase_array,nbase_all,L80,Project,SB_name,blc_tint,blc_ntunings,bp_time,flux_time,phase_time,pol_time,check_time,target_time_tot,time_tot,cal_time,mfssize,wsu_cubesize_stepped2,wsu_productsize_early_stepped2,wsu_datarate_early_stepped2_typical,wsu_visrate_early_stepped2_typical,wsu_datavol_early_stepped2_typical_target_tot,wsu_datavol_early_stepped2_typical_cal,wsu_datavol_early_stepped2_typical_total,wsu_nvis_early_stepped2_typical_target_tot,wsu_nvis_early_stepped2_typical_cal,wsu_nvis_early_stepped2_typical_total,wsu_productsize_later_2x_stepped2,wsu_datarate_later_2x_stepped2_typical,wsu_visrate_later_2x_stepped2_typical,wsu_datavol_later_2x_stepped2_typical_target_tot,wsu_datavol_later_2x_stepped2_typical_cal,wsu_datavol_later_2x_stepped2_typical_total,wsu_nvis_later_2x_stepped2_typical_target_tot,wsu_nvis_later_2x_stepped2_typical_cal,wsu_nvis_later_2x_stepped2_typical_total,wsu_productsize_later_4x_stepped2,wsu_datarate_later_4x_stepped2_typical,wsu_visrate_later_4x_stepped2_typical,wsu_datavol_later_4x_stepped2_typical_target_tot,wsu_datavol_later_4x_stepped2_typical_cal,wsu_datavol_later_4x_stepped2_typical_total,wsu_nvis_later_4x_stepped2_typical_target_tot,wsu_nvis_later_4x_stepped2_typical_cal,wsu_nvis_later_4x_stepped2_typical_total,blc_cubesize,blc_cubesize_sum,blc_productsize,blc_datarate_typical,blc_visrate_typical,blc_datavol_typical_target_tot,blc_datavol_typical_cal,blc_datavol_typical_total,blc_nvis_typical_target_tot,blc_nvis_typical_cal,blc_nvis_typical_total,cycle_info,imsize_mit,wsu_cubesize_stepped2_mit,wsu_mfssize_mit,wsu_productsize_early_stepped2_mit,plversion,procedure,nant,pl_totaltime,pl_imgtime,pl_cubetime,pl_aggtime,pl_fctime,nEB,npt,nscan,reptgt,allowedcubesize,allowedcubelimit,predcubesize,mitigatedcubesize,allowedprodsize,initialprodsize,mitigatedprodsize,mitigated,mit_nbins,mit_hm_imsize,mit_hm_cell,mit_field,mit_spw,pl_caltime
,,,,,,,,,,,,deg,arcsec,,,arcsec,arcsec,,,kHz,GHz,km / s,,,GHz,GHz,GHz,,GHz,GHz,GHz,GHz,,,,kHz,,km / s,kHz,,km / s,kHz,,km / s,s,,,,,,,,,,,m,,,s,,s,s,s,s,s,s,s,s,Gbyte,Gbyte,Gbyte,Gbyte / s,Gvis / h,Gbyte,Gbyte,Gbyte,Gvis,Gvis,Gvis,Gbyte,Gbyte / s,Gvis / h,Gbyte,Gbyte,Gbyte,Gvis,Gvis,Gvis,Gbyte,Gbyte / s,Gvis / h,Gbyte,Gbyte,Gbyte,Gvis,Gvis,Gvis,Gbyte,Gbyte,Gbyte,Gbyte / s,Gvis / h,Gbyte,Gbyte,Gbyte,Gvis,Gvis,Gvis,,,Gbyte,Gbyte,Gbyte,,,,h,h,h,h,h,,,,,Gbyte,Gbyte,Gbyte,Gbyte,Gbyte,Gbyte,Gbyte,,,,,,,h
str22,str14,str26,str3,str105,str27,str52,int64,int64,int64,float64,float64,float64,float64,str1,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str14,str26,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float6

In [40]:
?np.extend

Object `np.extend` not found.


In [41]:
myrows = []

In [43]:
test = wsu_all[0]['time_tot']

In [44]:
test

<Quantity 3568.32 s>

In [47]:
myrows.extend(test)

TypeError: 'Quantity' object with a scalar value is not iterable

In [46]:
myrows

[<Quantity 3568.32 s>]